## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,% Humidity,% Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.73,-155.09,80.33,83,90,11.50,moderate rain
1,1,Pevek,RU,69.70,170.31,38.44,86,97,9.69,overcast clouds
2,2,Bambous Virieux,MU,-20.34,57.76,68.22,94,75,16.11,moderate rain
3,3,Anjozorobe,MG,-18.40,47.87,48.52,95,39,3.40,scattered clouds
4,4,Hithadhoo,MV,-0.60,73.08,80.74,78,61,9.01,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,% Humidity,% Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.73,-155.09,80.33,83,90,11.50,moderate rain
4,4,Hithadhoo,MV,-0.60,73.08,80.74,78,61,9.01,broken clouds
11,11,Salalah,OM,17.02,54.09,77.09,100,90,9.22,drizzle
15,15,Naukot,PK,24.86,69.40,83.25,77,100,23.44,overcast clouds
16,16,Hamilton,US,39.18,-84.53,77.88,79,1,0.00,clear sky
24,24,Fortuna,US,40.60,-124.16,77.47,92,90,4.61,overcast clouds
26,26,Mitsamiouli,KM,-11.38,43.28,75.42,68,2,8.03,clear sky
31,31,Saint Marys,US,30.73,-81.55,79.23,74,1,4.00,clear sky
35,35,Wajima,JP,37.40,136.90,83.08,69,75,5.75,broken clouds
38,38,Mayo,US,38.89,-76.51,82.13,87,1,1.01,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             198
City                   198
Country                198
Lat                    198
Lng                    198
Max Temp               198
% Humidity             198
% Cloudiness           198
Wind Speed             198
Current Description    198
dtype: int64

In [6]:
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
% Humidity             0
% Cloudiness           0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,80.33,moderate rain,19.73,-155.09,
4,Hithadhoo,MV,80.74,broken clouds,-0.60,73.08,
11,Salalah,OM,77.09,drizzle,17.02,54.09,
15,Naukot,PK,83.25,overcast clouds,24.86,69.40,
16,Hamilton,US,77.88,clear sky,39.18,-84.53,
24,Fortuna,US,77.47,overcast clouds,40.60,-124.16,
26,Mitsamiouli,KM,75.42,clear sky,-11.38,43.28,
31,Saint Marys,US,79.23,clear sky,30.73,-81.55,
35,Wajima,JP,83.08,broken clouds,37.40,136.90,
38,Mayo,US,82.13,clear sky,38.89,-76.51,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    #6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,80.33,moderate rain,19.73,-155.09,Hilo Hawaiian Hotel
4,Hithadhoo,MV,80.74,broken clouds,-0.60,73.08,Scoop Guest House
11,Salalah,OM,77.09,drizzle,17.02,54.09,HAMDAN PLAZA HOTEL SALALAH
15,Naukot,PK,83.25,overcast clouds,24.86,69.40,Danish Boot House
16,Hamilton,US,77.88,clear sky,39.18,-84.53,Six Acres Bed & Breakfast
...,...,...,...,...,...,...,...
652,Kokoda,PG,76.17,overcast clouds,-8.88,147.74,Daito Guest House
654,Puerto Escondido,MX,77.83,overcast clouds,15.85,-97.07,Aldea del Bazar Hotel and Spa
657,Avera,US,82.27,overcast clouds,33.19,-82.53,
662,Jaru,BR,82.31,overcast clouds,-10.44,-62.47,Hotel Paraná


In [21]:
hotel_df.count()

City                   198
Country                198
Max Temp               198
Current Description    198
Lat                    198
Lng                    198
Hotel Name             198
dtype: int64

In [39]:
# Fill in the empty rows with "NaN".
hotel_df['Hotel Name'].replace(' ', np.nan, inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,80.33,moderate rain,19.73,-155.09,Hilo Hawaiian Hotel
4,Hithadhoo,MV,80.74,broken clouds,-0.60,73.08,Scoop Guest House
11,Salalah,OM,77.09,drizzle,17.02,54.09,HAMDAN PLAZA HOTEL SALALAH
15,Naukot,PK,83.25,overcast clouds,24.86,69.40,Danish Boot House
16,Hamilton,US,77.88,clear sky,39.18,-84.53,Six Acres Bed & Breakfast
...,...,...,...,...,...,...,...
652,Kokoda,PG,76.17,overcast clouds,-8.88,147.74,Daito Guest House
654,Puerto Escondido,MX,77.83,overcast clouds,15.85,-97.07,Aldea del Bazar Hotel and Spa
657,Avera,US,82.27,overcast clouds,33.19,-82.53,NaN
662,Jaru,BR,82.31,overcast clouds,-10.44,-62.47,Hotel Paraná


In [42]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,80.33,moderate rain,19.73,-155.09,Hilo Hawaiian Hotel
4,Hithadhoo,MV,80.74,broken clouds,-0.60,73.08,Scoop Guest House
11,Salalah,OM,77.09,drizzle,17.02,54.09,HAMDAN PLAZA HOTEL SALALAH
15,Naukot,PK,83.25,overcast clouds,24.86,69.40,Danish Boot House
16,Hamilton,US,77.88,clear sky,39.18,-84.53,Six Acres Bed & Breakfast
...,...,...,...,...,...,...,...
643,Kloulklubed,PW,79.30,overcast clouds,7.04,134.26,Storyboard Beach Resort
652,Kokoda,PG,76.17,overcast clouds,-8.88,147.74,Daito Guest House
654,Puerto Escondido,MX,77.83,overcast clouds,15.85,-97.07,Aldea del Bazar Hotel and Spa
662,Jaru,BR,82.31,overcast clouds,-10.44,-62.47,Hotel Paraná


In [43]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))